<a href="https://colab.research.google.com/github/Caffeinboy/cnn/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#convert MNIST image files into tensor of 4D(#if image,height,width)
transform = transforms.ToTensor()

In [ ]:
#Train data
train_data = datasets.MNIST(root='/cnn_data', train=True, download=True, transform=transform)

In [ ]:
#Test data
test_data = datasets.MNIST(root='/cnn_data', train=False, download=True, transform=transform)

In [ ]:
train_data

In [ ]:
test_data

In [ ]:
pwd

In [ ]:
ls

In [ ]:
cd ../

In [ ]:
ls

In [ ]:
cd cnn_data

In [ ]:
ls

In [ ]:
cd ../

In [ ]:
ls

In [ ]:
cd content

In [ ]:
#create a small batch size image
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(train_data, batch_size=10, shuffle=False)

In [ ]:
#define our CNN model
#describe convolutional layers & its doing(2 layers)
conv1 = nn.Conv2d(1, 6, 3, 1)
conv2 = nn.Conv2d(6, 16, 3, 1)

In [ ]:
#grab 1 MNIST image/record
for i, (x_train,y_train) in enumerate(train_data):
  break

In [ ]:
x_train.shape

In [ ]:
x = x_train.view(1,1,28,28)

In [ ]:
#perform the 1st convolution
x = x_train.view(1,1,28,28) # Re-initialize x with the original input shape
x = F.relu(conv1(x)) #Rectified linear unit for our activation function

In [ ]:
#single image,6 is the filters asked for 26x26
x.shape

In [ ]:
#pass thru the pooling
x = F.max_pool2d(x, 2,2)#kernal of 2 & stride of 2

In [ ]:
x.shape #26 / 2 = 13

In [ ]:
# do 2nd convolusional layer
x = F.relu(conv2(x))

In [ ]:
x.shape #loose padding since we didn't set em so lose 2 px

In [ ]:
#pooling layer
x = F.max_pool2d(x, 2,2)

In [ ]:
x.shape # 11 / 2 =5.5 --> 5

In [ ]:
# Model class
class convolusionalNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 6, 3, 1)
    self.conv2 = nn.Conv2d(6, 16, 3, 1)
    #fully connected layer
    self.fc1 = nn.Linear(5*5*16, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = F.max_pool2d(x, 2,2) # 2x2 kernal & stride of 2

    #second pass
    x = F.relu(self.conv2(x))
    x = F.max_pool2d(x, 2,2)

    #Re-view to flatten it out
    x = x.view(-1, 5*5*16)

    #fully connected layers
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    return F.log_softmax(x, dim=1)

In [ ]:
#create an instance of our model
torch.manual_seed(101)
model = convolusionalNetwork()
model

In [ ]:
#loss function optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.008) #smaller learning rate,longer it takes

In [ ]:
import time
start_time = time.time()

#create variables to track things
epochs = 30
train_losses = []
test_losses = []
train_correct = []
test_correct = []


#For loop of epochs
for i in range(epochs):
  trn_corr = 0
  tst_corr = 0


  #Train
  for b,(x_train, y_train) in enumerate(train_loader):
    b += 1
    y_pred = model(x_train) #get predicted values
    loss = criterion(y_pred, y_train) #off the compare prediction
    predicted = torch.max(y_pred.data, 1)[1]#add the no of correct prediction
    batch_corr = (predicted == y_train).sum() #how many got correct
    trn_corr += batch_corr


    #update our parameters
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    train_losses.append(loss)
    train_correct.append(trn_corr)


  #print out some results
  if b%60 == 0:
     print(f'epoch: {i} batch: {b} loss: {loss.item()}')

  train_losses.append(loss)
  train_correct.append(trn_corr)

  #Test
  with torch.no_grad():
   for b,(x_test, y_test) in enumerate(test_loader):
    y_val = model(x_test)
    predicted = torch.max(y_val.data, 1)[1]
    tst_corr += (predicted == y_test).sum()

  loss = criterion(y_val, y_test)
  test_losses.append(loss)
  test_correct.append(tst_corr)

  current_time = time.time()
  total = current_time - start_time
  print(f'Training tool: {total/60}minutes!')

In [ ]:
# Only convert if they're tensors, otherwise leave them
plt.plot([t.item() for t in train_losses], label="Training Loss")
plt.plot([t.item() for t in test_losses], label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss per Epoch")
plt.legend()
plt.show()

In [ ]:
#graph the accuracy end of epoch
plt.plot([t/600 for t in train_correct], label='Training Accuracy')
plt.plot([t/100 for t in test_correct], label='Testing Accuracy')
plt.title('Training & Testing Accuracy')
plt.legend()